In [ ]:
import re

In [ ]:
pattern = 'phone'
text = "The person's phone number is 408-555-1234. Call soon!"

In [ ]:
re.search(pattern,text)

In [ ]:
pattern = 'NOT IN TEXT'

In [ ]:
re.search(pattern,text)

In [ ]:
pattern = 'phone'

In [ ]:
match = re.search(pattern,text)

In [ ]:
match.span()

In [ ]:
text = "my phone is a new phone"

In [ ]:
matches = re.findall(pattern,text)

In [ ]:
type(matches)

In [ ]:
matches

In [ ]:
[match.group() for match in re.finditer(pattern,text)]

In [ ]:
match.group()

In [ ]:
phone = re.search(r'\d\d\d-\d\d\d-\d\d\d\d', text)

In [ ]:
phone.group()

In [ ]:
text = '.MDD_a_bArrW4000[1877]'

In [ ]:
item = 1877

In [ ]:
pattern = rf".MDD_a_bArrW\d{{4}}\[{item}\]"
print(pattern)

In [ ]:
match = re.search(pattern,text)

In [ ]:
import re

In [ ]:
def search_variable(data : list, var_name : str) -> str:
    pattern = rf".MDD_a_bArr\D\d{{4}}\[{var_name}\]"
    if re.search(pattern,data):
        return True

In [ ]:
search_variable(text,1877)

In [ ]:
re.search(r"man|woman", "This woman was there")

In [ ]:
re.findall(r'.at', 'The cat in the hat sat here')

In [ ]:
re.findall(r'\S+at', 'The bat went splat')

In [ ]:
re.findall(r'\d$','This ends with a number 2')

In [ ]:
re.findall(r'^\d','1 us the lonekiest number')

In [ ]:
phrase = "there are 3 numbers 34 inside 5 this sentence."

In [ ]:
re.findall(r'[^\d]',phrase)

In [ ]:
re.findall(r'[^\d]+',phrase)

In [ ]:
test_phrase = 'This is a string! But it has punctuation. How can we remove it?'

In [ ]:
clean = ' '.join(re.findall(r'[^!.? ]+',test_phrase))

In [ ]:
clean

In [ ]:
re.search(r'cat|dog', 'The cat is here')

In [ ]:
re.findall(r'..at','The cat in the hat is sat here')

In [ ]:
3

In [ ]:
def func_one(n):
    return [str(num) for num in range(n)]

In [ ]:
func_one(10)

In [ ]:
def func_two(n):
    return list(map(str,range(n)))

In [ ]:
func_two(10)

In [ ]:
import time

In [ ]:
#current time before
start_time = time.time()
#run code
result =func_two(100000000)
#current time after running code
end_time = time.time()
# eldpsed time
elapsed_time = end_time - start_time

print(elapsed_time)

In [ ]:
import datetime

class TimeStamp:
    """
    Обернул в класс методы которые возвращают timestamp
    Может вернуть дату + время , а может только время
    """
    def get_time_stamp(self) -> str:
        return datetime.datetime.today().strftime('%H:%M:%S')

    def get_datetime_stamp(self) -> str:
        return datetime.datetime.today().strftime('%c')


class PlcVariable:
    """
    This class contains plc.variable information:
    path, status, value, name.
    """

    def __init__(self,name:str,value:int,plc) -> None:
        self.name = name
        self.value = value
        self.plc = plc
        self.history = [self.value]


    def __str__(self) -> str:
        return f"Name: {self.name}, Value: {self.value}"


    def diff_checking(func):
        """ Декоратор в котором сверяется прошлое значение переменной и нынешнее"""
        def checnking(self):
            prev_value = self.value
            func(self)
            if prev_value != self.value:
                print(f'{TimeStamp().get_time_stamp()} - Value of {self.name} has been changed from {prev_value} to {self.value}')
                self.history.append(self.value) # Добавляем новое значение в history, типа отслеживаем изменение
                print(f'History: {self.history}<--Now')
        return checnking


    def get_name(self):
        return self.name


    def get_value(self):
        return self.value

    @diff_checking
    def update_value(self):
        self.value = self.plc.read_by_name(self.name,pyads.PLCTYPE_INT)  


class VariableContainer:
    """
    В контейнер собираем все переменные. Обновляем их и проверяем статус.
    """
    def __init__(self,plc) -> None:
        self.container = []
        self.plc = plc

    def __str__(self) -> str:
        """ Container str representation """
        return f'Container:\n {self.container}'

    def get_names(self)-> list:
        """ Return a list of variables names\paths are contained in the container """
        return [variable.name for variable in self.container]

    def add_to_container(self,plc_variable):
        """ Add new PlcVariable instance into container"""
        self.container.append(plc_variable)

    def show_container(self) -> None:
        """
        Print the contents of the container
        It looks like: "'Name' 100"
        """
        for plc_variable in self.container:
            print(plc_variable.name,plc_variable.value)
    
    def update_container(self):
        """
        ***NOT USED***
        Refresh all values in the container's PLC variables
        Use pyads.Connection.read_list_by_name - method
        But may be it's better to read values from the plc one by one.
        TODO: check it. Compare these two ways.
        """
        if self.container:
            values = self.plc.read_list_by_name(container.get_names())
            self.container = []
            for name in values:
                self.add_to_container(PlcVariable(
                                        name=name,
                                        value=values[name],
                                        plc=self.plc))
            return self.container
    
    def update_variables(self)-> None:
        """ Update all the conteiner's variables one by one with PlcVariable.update_value()"""
        if self.container:
            for item in self.container:
                item.update_value()


class CenterLine:
    def __init__(self,name,setpoint,plc_variable : PlcVariable, maximum : int=None, minimum : int=None) -> None:
        self.name = name
        self.setpoint = setpoint
        self.plc_variable = plc_variable 
        self.status = False

        if not maximum and not minimum:
            self.maximum = self.setpoint
            self.minimum = self.setpoint   # Если не задали пределы, то пределы пусть будут равны значению.
        else:
            self.maximum = maximum
            self.minimum = minimum

        self._status()

    def __str__(self)-> str:
        return f"CL-'{self.name}'.Setpoint-{self.setpoint}. PLC Value-{self.get_plc_value()}. Status-{self.get_str_status()}"
    

    def get_plc_value(self)-> int:
        return int(self.plc_variable.value)


    def _status(self) -> None:
        if self.get_plc_value() == self.setpoint or self.in_range():
            self.status = True
        else:
            self.status = False


    def get_str_status(self) -> str:
        if self.status: return 'OK'
        else: return 'NOK'


    def in_range(self) -> bool:
        return self.minimum <= self.get_plc_value() <= self.maximum        

position_var = PlcVariable('MAIN/POSITION',100,plc)
cl = CenterLine('Position',100,position_var,minimum=90,maximum=105)
print(cl)
position_var.value =88
cl._status()
print(cl)
position_var.value = 103
cl._status()
print(cl)

       

In [ ]:
class CenterLine:
    def __init__(self,name,setpoint,plc_variable : PlcVariable, maximum : int=None, minimum : int=None) -> None:
        self.name = name
        self.setpoint = setpoint
        self.plc_variable = plc_variable 

        if not maximum and not minimum:
            self.maximum = self.setpoint
            self.minimum = self.setpoint   # Если не задали пределы, то пределы пусть будут равны значению.
        else:
            self.maximum = maximum
            self.minimum = minimum

        self.status = self.set_status()

    def __str__(self):
        return f'CL - {self.name}.Setpoint Value - {self.setpoint}. PLC Value - {self.get_plc_value()}. Status - {self.get_str_status()} '
    

    def get_plc_value(self):
        return int(self.plc_variable.value)


    def set_status(self):
        if self.get_plc_value() == self.setpoint or self.in_range():
            self.status = True
        else:
            self.status = False


    def get_str_status(self):
        if self.status: return 'OK'
        else: return 'NOK'


    def in_range(self):
        return self.minimum <= self.get_plc_value() <= self.maximum
                


In [ ]:
position_var = PlcVariable('MAIN/POSITION',100,plc)
cl = CenterLine('Position',100,position_var,minimum=90,maximum=105)
print(cl)
position_var.value =88
cl.set_status()
print(cl)



In [ ]:
def get_var_list(plc)-> list: 
    try:
        symbols=plc.get_all_symbols()
        variables = []

        for s in symbols:
            if s.name[0] != '.':
                variables.append(s.name)

        return variables
    except:
        print('He have an error while we read stuff')


In [ ]:
def get_vars_from_plc(plc,var_list) -> list:
    return plc.read_list_by_name(var_list)

In [ ]:
from time import sleep
from IPython.display import clear_output
import pyads

remote_ip = '192.168.1.177'

remote_ads = '192.168.1.177.1.1'

with pyads.Connection(remote_ads, pyads.PORT_TC2PLC1, remote_ip) as plc:
    #variable_list = get_var_list(plc)

    variable_list = [
        'MAIN.CENTER_DRUM_ENGAGE_POSITION',
        'MAIN.LEFT_DRUM_ENGAGE_POSITION',
        'MAIN.RIGHT_DRUM_SETPOINT_POSITION',
        'MAIN.SPEED_SETPOINT',
        'MAIN.TEMPERATURE_SETPOINT',
        'MAIN.WORK_POSITION'
    ]
    dump = get_vars_from_plc(plc,variable_list)    
    container = VariableContainer(plc)
    
    for v in dump:
        container.add_to_container(PlcVariable(name=v,value=dump[v],plc=plc))

    while True:
        container.update_variables()
        #container.show_container()
        sleep(1)
    

In [7]:
import datetime
datetime.datetime.today().strftime('%H:%M:%S')

'18:37:12'

In [8]:
class TimeStamp:

    def get_time_stamp(self):
        return datetime.datetime.today().strftime('%H:%M:%S')

    def get_datetime_stamp(self):
        return datetime.datetime.today().strftime('%c')

In [9]:
TimeStamp().get_time_stamp()

'18:37:14'

In [10]:
class Test:

    def __init__(self) -> None:
        self.counter = 0
        self.init = self.counter + 1

    def __str__(self):
        return str(self.init)

    def get_init(self):
        self
        return self.init

In [11]:
t = Test()
t.get_init()
t.get_init()

1